All-in-One rearing analyzer


 by max vaglica
https://github.com/Guillermo-Hidalgo-Gadea/UQOAB/blob/main/First%20DLC%20project/.ipynb_checkpoints/partII-checkpoint.ipynb



Run and restart runtime when prompted
 ,make sure runtime type is 'GPU'



# INITIALIZE DEEPLABCUT SESSION

Next three code blocks MUST be run everytime no matter whether you are training a new network, scoring videos with a trained network, or extracting new outlier frames.

In [ ]:
!pip install statsmodels==0.13.1
!pip install deeplabcut

In [ ]:
!pip install tables==3.5.1
!pip install tensorboard==2.7.0
!pip install tensorflow==2.7.0

After restarting the runtime, Import all and mount the memlab@iu.edu 
google drive. You may need to close and reopen your browser to login. Copy the code after allowing access to the google drive & paste it into the box.

In [ ]:
%tensorflow_version 2.x
import os
os.environ["DLClight"]="True"
import deeplabcut
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
ProjectFolderName = 'all_combined_tracker-aes-2022-06-22'
VideoType = 'mp4' 
#videofile_path = ['/content/drive/My Drive/Projects/DeepLabCut/'+ProjectFolderName+'/videos/']
path_config_file = '/content/drive/My Drive/Projects/DeepLabCut/'+ProjectFolderName+'/config.yaml'
testVideo_path = ['/content/drive/My Drive/Projects/DeepLabCut/'+ProjectFolderName+'/videos/']
analyze_video_path = ['/content/drive/My Drive/Projects/DeepLabCut/jump_analysis_vids/combined_tracker_squash']
second_analyze_video_path = ['/content/drive/My Drive/Projects/DeepLabCut/jump_analysis_vids/combined_tracker']
!ls '/content/drive/My Drive/Projects/DeepLabCut/all_combined_tracker-aes-2022-06-22/config.yaml'

# TRAIN A NEW NETWORK

Be sure that the variables above match the path of your training videos

The following code will create the training dataset for the network.

"After running this script the training dataset is created and saved in the project directory under the subdirectory 'training-datasets'

This function also creates new subdirectories under dlc-models and appends the project config.yaml file with the correct path to the training and testing pose configuration file. **These files hold the parameters for training the network.**" 

**The pose_cfg.yaml file will store all configuration for training, edit these parameters to change training settings

In [ ]:
deeplabcut.create_training_dataset(path_config_file, net_type='resnet_50', augmenter_type='imgaug')

Below begins actual training on the network. You want at least 400,000 iterations. saveiters will save a network every 4500 iterations, this can be changed to any amount. Trained networks are saved as 3 'snapshot' files in the same directory. Change the initial learning to a previous snapshot to continue training from save, in the same format I have done in the yaml file. A checkpoint file is updated and saved for the entire training. All 4 of these files are needed to evaluate the network and analyze new videos.

In [ ]:
deeplabcut.train_network(path_config_file, shuffle=1,displayiters=100,saveiters=2000)

# EVALUATE A TRAINED NETWORK

This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images) and stores the results as .csv file in a subdirectory under evaluation-results

In [ ]:
%matplotlib notebook
deeplabcut.evaluate_network(path_config_file,plotting=False)

To store the coordinates for each body part in a csv file, run the store_coordinates(video_name) function. csv will be located with rest of file.

# REFINE A TRAINED NETWORK


In [ ]:
deeplabcut.extract_outlier_frames(path_config_file,testVideo_path,videotype='mp4', automatic=True)

**That was step 1. Now. . .**
Now you will need to go back to a computer with GUI capability (i.e., not Colab; e.g., Skywalker or your laptop) and 'refine_labels'. This is done with the GUI. Specifically-


*   Open the GUI
*   Select the config file for this project and load the project
*   Click on the last tab
*   Click LAUNCH
*   Click on 'Load labels' (button bottom left of pop-up)
*   Choose a folder in the 'labeled_videos' folder, select the 'machinelabels*.h5' file.
*   Continue like you were labeling the videos the first time. There will already be labels placed by the network. Most of the time one or more will be wrong. Your job is to fix them. Be sure to hit save (often and) when you're done.




In [ ]:
deeplabcut.merge_datasets(path_config_file)

In [ ]:
deeplabcut.create_training_dataset(path_config_file, net_type='resnet_50', augmenter_type='imgaug')

If you want to pick up training from where you left off before - 
*    Find the train subdirectory within the dlc-models directory and look for the latest snapshot. Open the pose_cfg.yaml file within the same folder and edit the parameter init_weights: '-snapshot-10000' without any filetype ending
*   It probably previously said /Users/ehren/opt/anaconda3/envs/DEEPLABCUT_M1/lib/python3.7/site-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt

In [ ]:
deeplabcut.train_network(path_config_file, shuffle=1,displayiters=100,saveiters=2000)

# PUT THAT NETWORK TO WORK!!

You did it! Really! If you made it this far, your network should be working nicely. If not, repeat the last two blocks until you are happy with the way it is working. Once you are happy with it, now you can use the following commands to label your videos. . .

If you're wondering if it is working well, try the following lines out. Look at the labeled videos and analyze the csv file coordinates. If the video is accurately labelled and the coordinates in the csv file are stable (i.e., don't jump around) then your network is likely working well! Just watch for when the rat does strange things, the networks struggle most with these. . . 

**Instructions**:


*   Put your videos that you want analyzed into the folder pointed to by 'testVideo_path'. This is likely the folder called 'videos' in your project folder. 
*   Call the following two commands. Technically, only the first is required. This creates the csv file that matlab will use to analyze the data. The second, however, is useful if you want to preview how well the network labeled the video: it will re-create the video with little dots superimposed on it showing where the network thought each body part was.

*   These commands will put their output files (the csv and labeled video) into the same folder as where you put your videos to be analyzed. 
*    Once each video has been processed, you can remove them from the Google Drive folder and put them back in the appropriate place on the Dropbox file system (e.g., in your project folder or with the original video).

In [ ]:
deeplabcut.analyze_videos(path_config_file,testVideo_path, videotype='mp4', save_as_csv=True)

In [ ]:
deeplabcut.create_labeled_video(path_config_file,testVideo_path, videotype='mp4',draw_skeleton=False,save_frames=False)